In [1]:
# Chứng minh việc loại bỏ dữ liệu rác (Outliers nhiễu và cột vô nghĩa) giúp mô hình tránh bị "đánh lạc hướng".

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
url = "https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv"
df = pd.read_csv(url)

In [4]:
df_dirty = df.copy()
df_dirty["noise_col"] = 999 
df_dirty.loc[0, ['sepal_length', 'sepal_width']] = [50.0, 50.0]

In [5]:
X = df_dirty.drop("species", axis=1)
y = df_dirty["species"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
model_dirty = DecisionTreeClassifier(random_state=42)
model_dirty.fit(X_train, y_train)
acc_dirty = accuracy_score(y_test, model_dirty.predict(X_test))

In [7]:
# A) Loại bỏ cột vô nghĩa
const_cols = [col for col in X_train.columns if X_train[col].nunique() <= 1]
X_train_cleaned = X_train.drop(columns=const_cols)
X_test_cleaned = X_test.drop(columns=const_cols)

In [8]:
# B) Loại bỏ Outlier (Z-score đơn giản)
# (Chỉ thực hiện trên tập Train để tránh rò rỉ thông tin)
z_scores = np.abs((X_train_cleaned - X_train_cleaned.mean()) / X_train_cleaned.std())
X_train_final = X_train_cleaned[(z_scores < 3).all(axis=1)]
y_train_final = y_train[X_train_final.index]

In [9]:
model_clean = DecisionTreeClassifier(random_state=42)
model_clean.fit(X_train_final, y_train_final)
acc_clean = accuracy_score(y_test, model_clean.predict(X_test_cleaned))

In [10]:
print(f"Accuracy TRƯỚC Cleaning: {acc_dirty:.4f}")
print(f"Accuracy SAU Cleaning: {acc_clean:.4f}")

Accuracy TRƯỚC Cleaning: 1.0000
Accuracy SAU Cleaning: 1.0000
